In [ ]:
# Example 21: live observation of single satellite
# SPDX-FileCopyrightText: Copyright (C) 2023 Andreas Naber <annappo@web.de>
# SPDX-License-Identifier: GPL-3.0-only

%matplotlib widget

import asyncio
from rtlsdr import RtlSdr
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
from scipy.fft import fft, ifft
import gpslib_tutorial as gpslib
import time
import datetime
import json
from collections import deque
import linecache
import sys

# -------- floating point types used in arrays ---------

MY_FLOAT = np.float32
MY_COMPLEX = np.complex64


# --------- global variables ----------

SAMPLE_RATE = 2.048E6
SDR_FREQCORR = -2
SDR_CENTERFREQ = 1575.42e6      # GPS L1 band
SDR_GAIN = 49.9

MEAS_RUNNING = True      
PHASE_LOCKED = False     

STATUS_MSG = ''          

SMP_TIME = 0             
MS_TIME = 0
    
N_CYC = 32               
CODE_SAMPLES = 2048      
NGPS = N_CYC*CODE_SAMPLES
CORR_AVG = 8             
DLF = 1024 // N_CYC      

SEC_TIME = np.linspace(1,NGPS,NGPS,endpoint=True)/SAMPLE_RATE  
FRAME_LIST = []
COPH_LIST = [] 

IT_SWEEP = 16
MIN_FREQ = -5000.0
MAX_FREQ = +5000.0
STEP_FREQ = 200           
START_FREQ = MIN_FREQ
CORR_MIN = 8

DF_GAIN1 = 10 
DF_GAIN2 = 1
LOCKED_THRES = 0.3
    
SAVE_RESULTS = False           # save COPH_LIST and FRAME_LIST
SAVE_DATE = '150624'           # used in filename together with SAT_NO

SAT_NO = 3
SHOW_FRAMES = True
MEAS_TIME = 600                # time in seconds

# Precomputed FFT of interpolated cacode 
fftCacode = fft(gpslib.GPSCacode(SAT_NO,CODE_SAMPLES))


# ------- Exception handling & Output -------------------

EXC = ''

def printException():
    global EXC
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    EXC = ('EXCEPTION IN ({}, LINE {} "{}"): {}'\
           .format(filename, lineno, line.strip(), exc_obj))        
       
        
# -------- keyboard events ---------------

def onKeypress(event):
    global STATUS_MSG
    global MEAS_RUNNING
    sys.stdout.flush()
    if event.key in ['q','Q']:
        MEAS_RUNNING = False
        STATUS_MSG = 'q pressed'

# ------- data buffer -----------------


MAXBUFSIZE = 8                       
BUFFER = deque([],maxlen=MAXBUFSIZE)
NBUF = 0                             
BUFSKIP = 0

def pushToBuffer(data):
    global BUFFER
    global NBUF
    global BUFSKIP

    if NBUF >= MAXBUFSIZE:
        BUFFER.clear()
        NBUF = 0
        BUFSKIP += MAXBUFSIZE
        
    BUFFER.append(data)
    NBUF += 1
    
    
def pullFromBuffer(n):
    global BUFFER
    global NBUF
    global BUFSKIP

    try:
        data = BUFFER.popleft()
        NBUF -= 1
        skip = BUFSKIP
        BUFSKIP = 0
    except IndexError:
        data = []
        skip = 0
    
    return data,n,skip


# -------- JSON Encoder for numpy ----------------------

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)
        
    
# ----------- Save Positions, Frames and CodePhases -----------------------           
            
def saveResults(saveDate,satNo):                    
    try:
        with open(f'CP_{saveDate}_{satNo}.json','w') as file:
            json.dump(COPH_LIST,file,indent=2,cls=MyEncoder)
        with open(f'SF_{saveDate}_{satNo}.json','w') as file:
            json.dump(FRAME_LIST,file,indent=2,cls=MyEncoder)
                        
    except:
        printException() 
        
    
#------------- Function evalGpsBits (uses GpsSubframe) --------------------    

def evalGpsBits(gpsBits,gpsBitsSmpTime):    
    Result = []
    if len(gpsBits) < 300:
        return Result, gpsBits, gpsBitsSmpTime

    gb = np.copy(gpsBits)
    preamble = np.array([1,-1,-1,-1,1,-1,1,1],dtype=np.int8)
    bitsCorr = np.correlate(gb,preamble,mode='same')
    locPreamble =[]
    for i in range(len(bitsCorr)):
        if abs(bitsCorr[i])==8:
            locPreamble.append(i-4)  

    start = 0
    if len(locPreamble) > 0: 
        gb[gb==-1] = 0               
        lpIndex = 0
        start = locPreamble[lpIndex]
        ok = True
        while ok and start+300 < len(gb):
            sf = gpslib.Subframe()  
            if sf.Extract(gb[start:start+300]) == 0:
                ST = gpsBitsSmpTime[start]
                if sf.ID == 1:
                    res = {'ID': sf.ID,
                           'tow': sf.tow,
                           'weekNum': sf.weekNum,
                           'satAcc': sf.satAcc,
                           'satHealth': sf.satHealth,
                           'Tgd': sf.Tgd,
                           'IODC': sf.IODC,
                           'Toc': sf.Toc,
                           'af2': sf.af2,
                           'af1': sf.af1,
                           'af0': sf.af0,
                           'ST': ST}
                elif sf.ID == 2:
                    res = {'ID': sf.ID,
                           'tow': sf.tow,
                           'Crs': sf.Crs,
                           'deltaN': sf.deltaN,
                           'M0': sf.M0,
                           'Cuc': sf.Cuc, 
                           'IODE2': sf.IODE2,
                           'e': sf.e,
                           'Cus': sf.Cus,
                           'sqrtA': sf.sqrtA,
                           'Toe': sf.Toe,
                           'ST': ST}
                elif sf.ID == 3:
                    res = {'ID': sf.ID,
                           'tow': sf.tow,
                           'Cic': sf.Cic,
                           'omegaBig': sf.omegaBig,
                           'Cis': sf.Cis,
                           'i0': sf.i0, 
                           'IODE3': sf.IODE3,
                           'Crc': sf.Crc,
                           'omegaSmall': sf.omegaSmall,
                           'omegaDot': sf.omegaDot,
                           'IDOT': sf.IDOT,
                           'ST': ST}
                elif sf.ID == 4 or sf.ID == 5:
                    res = {'ID': sf.ID,
                           'tow': sf.tow,
                           'ST': ST}
                Result.append(res)
                start += 300  
            else:
                ok = False
                while not ok and lpIndex<len(locPreamble)-1:
                    lpIndex += 1
                    s = locPreamble[lpIndex]
                    ok = s > start                    
                if ok:
                    start = s                

    return Result, gpsBits[start:], gpsBitsSmpTime[start:]



# --------------------------

GPSBITS    = np.array([],dtype=np.int8) 
GPSBITS_ST = np.array([],dtype=np.int64)

def evalEdges(edges):                  
    global GPSBITS
    global GPSBITS_ST
    
    frameData = []
    if len(edges) > 2:            
        edges,bits,bitsSmpTime = logicalBits(edges)       
        GPSBITS = np.append(GPSBITS,bits)
        GPSBITS_ST = np.append(GPSBITS_ST,bitsSmpTime)   
        frameData,GPSBITS,GPSBITS_ST = evalGpsBits(GPSBITS,GPSBITS_ST) 

    return edges,frameData        


def logicalBits(edges):  
    bits = []
    bitsSmpTime = []
    lastSign = edges[0]
    n = len(edges)

    if n > 2:
        t1,st1 = edges[1]
        for i in range(2,n):
            t2,st2 = edges[i]
            m,r = np.divmod(t2-t1,20)
            if r > 17:
                m += 1
            if m > 0:                
                bits += [lastSign]*m
                bitsSmpTime += [st1]
                bitsSmpTime += [0]*(m-1) 
            t1 = t2
            st1 = st2
            lastSign = -lastSign        
        edges = [lastSign,edges[-1]]

    bits = np.asarray(bits,dtype=np.int8) 
    bitsSmpTime = np.asarray(bitsSmpTime,dtype=np.int64)

    return edges,bits,bitsSmpTime
    
    
# --------------------------

def bitPlotData(msTime,edges,n):
    bd = np.zeros(n,dtype=np.int8)
    t1 = msTime                   
    t0 = t1 - n + 1               
    firstSign = edges[0]

    if len(edges) == 1:
        bd[:] = firstSign
    elif len(edges) == 2:            
        t,st = edges[1]
        if t >= t0:
            bd[:t-t0] = firstSign
            bd[t-t0:n] = -firstSign
        else:
            bd[:] = -firstSign
    else:
        dt = edges[-1][0] - edges[1][0] 
        lastSign = (2*(len(edges) % 2) - 1) * edges[0] 
        bsc = [tms for tms,st in edges[1:]] 
        k1 = len(bsc)
        k0 = k1-1
        while k0>0 and bsc[k0]>t0:
            k0 -= 1
        ts = 0
        for t in reversed(bsc[k0:k1]):  
            te = min(ts+t1-t, n)
            bd[ts:te] = lastSign
            ts = te
            t1 = t
            lastSign = -lastSign
        if te < n:
            bd[te:n] = lastSign

    bd = np.flip(bd)

    return bd    

    
# --- Average data stream over 1ms ("low-pass filter") --------

PREV_SAMPLES = []  

def decodeData(data,cp,edges): 
    global PREV_SAMPLES
    global MS_TIME
    
    prevSign = (2*(len(edges) % 2) - 1) * edges[0]
        
    gpscacode = gpslib.GPSCacodeRep(SAT_NO,CODE_SAMPLES,N_CYC,cp)             
    y = gpscacode*data                                     

    nps = len(PREV_SAMPLES)           
    if nps > 0:
        y = np.append(PREV_SAMPLES,y)        
    ns = NGPS + nps 
    n0 = 0               
    n1 = nps + cp        
    if n1 == 0:
        n1 = CODE_SAMPLES
        ST = SMP_TIME
    else:        
        ST = SMP_TIME - CODE_SAMPLES + cp
               
    gpsData = []
    while n1 <= ns:
        m = np.mean(y[n0:n1])
        gpsData.append(m)
        
        if PHASE_LOCKED: 
            mSign = np.sign(m.real)
            if edges[0] == 0:    
                edges[0] = mSign 
                prevSign = mSign             
            else:
                if mSign != prevSign: 
                    edges.append((MS_TIME,ST+n0))
                    prevSign = mSign
            MS_TIME += 1                       
        
        n0 = n1
        n1 += CODE_SAMPLES
    gpsData = np.asarray(gpsData,dtype=MY_COMPLEX) 
    PREV_SAMPLES = y[n0:ns]
    
    return gpsData,edges


# -------- find frequency correction ----------    
    
def phaseLockedLoop(gpsData):
    global PHASE_LOCKED
    
    avg = 4       
    minDiff = 2.0                               

    n = len(gpsData)                            
    phase = np.arctan(gpsData.imag/gpsData.real) 

    dp = 0
    realPhase = np.copy(phase)
    for i in range(1,n):              
        delta = phase[i]-phase[i-1]
        dp -= np.sign(delta) if abs(delta) > minDiff else 0            
        realPhase[i] += dp*np.pi

    phaseOffset = np.mean(realPhase[-avg:])                               
    phaseDev = np.mean(realPhase)          
    if abs(phaseDev) < LOCKED_THRES:
        PHASE_LOCKED = True
        
    if PHASE_LOCKED:
        df = DF_GAIN2*phaseDev               
    else:
        df = DF_GAIN1*phaseDev             

    return df, phaseOffset

            
# ----- fit correlation maximum to improve codePhase -----
    
def fitMaxCorr(gpsCorr,mx):
    lgc = len(gpsCorr)
    ma = mx - 1 if mx > 0 else lgc-1
    mb = mx + 1 if mx < lgc-1 else 0

    x = np.asarray([-1,0,+1])
    y = np.asarray([gpsCorr[ma],gpsCorr[mx],gpsCorr[mb]])
    p = np.polyfit(x,y,2)
    if p[0] != 0.0:            
        fmx = -0.5*p[1]/p[0]   
        fitMax = mx + fmx      
    else:
        fitMax = -1.0

    return fitMax

# ------- Find maximum in correlation (delay) ---------

def findCodePhase(gpsCorr):
    mean = np.mean(gpsCorr)
    std = np.std(gpsCorr)
    delay = -1
    codePhase = -1.0
    
    mx = np.argmax(gpsCorr)
    normMaxCorr = (gpsCorr[mx]-mean)/std
    if normMaxCorr > CORR_MIN:
        delay = int(mx)
        codePhase = fitMaxCorr(gpsCorr,mx)   
             
    return delay,codePhase,normMaxCorr
    
# ------ cross-correlation -------------

def correlation(data,avg): 
    avg = min(avg,N_CYC)
    df = 0
    for i in range(avg):
        dfm = fft(data[i*CODE_SAMPLES:(i+1)*CODE_SAMPLES]) 
        df += dfm
    fftData = df/avg
    fftCorr = fftData*np.conjugate(fftCacode) 
    corr = np.abs(ifft(fftCorr))
    delay,codePhase,normMaxCorr = findCodePhase(corr)
    return corr,delay,normMaxCorr,codePhase


# ------  demodulation  -------------

def demodDoppler(data,freq,phase,N):                          
    factor = np.exp(-1.j*(phase+2*np.pi*freq*SEC_TIME[:N]))
    phase += 2*np.pi*freq*SEC_TIME[N-1]
    return factor*data[:N], np.remainder(phase,2*np.pi)


# ------- change doppler frequency within limits ---------
def confRange(freq):
    if freq > MAX_FREQ:
        freq -= MAX_FREQ - MIN_FREQ
    elif freq < MIN_FREQ:
        freq += MAX_FREQ - MIN_FREQ
    return freq

# -------- Sweep frequency -------------

def sweepFrequency(data,freq,n):
    delay = -1
    phase = 0
    k = 0
    N = CORR_AVG*CODE_SAMPLES    
    
    while delay<0 and k < n: 
        newdata,_ = demodDoppler(data,freq,phase,N)    
        gpsCorr,delay,normMaxCorr,_ = correlation(newdata,CORR_AVG)
        if delay<0:
            freq = confRange(freq+STEP_FREQ)
        k += 1
    sweepFreq = (delay < 0)

    return sweepFreq, freq

# ------------- plot text -----------

TOW_NO = -1
WEEK_NO = 0

def plotText(frameData):
    global TOW_NO
    global WEEK_NO
    if TOW_NO < 0:
        TOW_NO = frameData[-1]['tow']
    if frameData[-1]['ID'] == 1 and WEEK_NO == 0:
        WEEK_NO = frameData[-1]['weekNum']
        gpsTStr = gpslib.gpsTimeStr(TOW_NO,WEEK_NO)
        startL.set_text('START: %s' % (gpsTStr))
    for i,fd in enumerate(frameData[-MAXL:]):
        gpsT = gpslib.gpsTime(fd['tow'],0)
        dataL[i][keys[0]].set_text(gpsT.strftime('%H:%M:%S')) 
        dataL[i][keys[1]].set_text('%d' %(fd['tow']))
        dataL[i][keys[2]].set_text('%d' % (fd['ID']))
        dataL[i][keys[3]].set_text('%1.6f s' % (fd['ST']/SAMPLE_RATE))

# ------------- plot data -----------

def plotData(y1,y2):
    line1.set_ydata(y1.real)
    line2.set_ydata(y2)
    plt.draw() 
    
# ------- process data -------------------

async def processData():   
    global MEAS_RUNNING
    global SMP_TIME
    global FRAME_LIST
    global COPH_LIST
    
    edges = [0]
    gpsPlot = np.zeros(DLF*N_CYC,dtype=MY_COMPLEX)   
    bitPlot = np.zeros(DLF*N_CYC,dtype=MY_FLOAT)
    sweepAllFreq = True
    dopplerPhase = 0
    dopplerFreq = START_FREQ
    cp = 0
    noStream = 0
    try:
        while MEAS_RUNNING:
            if NBUF >= 1:
                data,n,skip = pullFromBuffer(NGPS)
                SMP_TIME += NGPS + skip                   
                if sweepAllFreq:
                    sweepAllFreq,dopplerFreq \
                        = sweepFrequency(data,dopplerFreq,IT_SWEEP)
                else:    
                    data,dopplerPhase \
                        = demodDoppler(data,dopplerFreq,dopplerPhase,NGPS)    
                    gpsCorr,delay,normMaxCorr,codePhase \
                        = correlation(data,CORR_AVG)               
                    if delay >= 0:
                        cp = delay 
                    gpsData,edges = decodeData(data,cp,edges)
                    bitData = bitPlotData(MS_TIME,edges,len(gpsData))       

                    gpsPlot = np.concatenate((gpsPlot[N_CYC:DLF*N_CYC],
                                              gpsData))   
                    bitPlot = np.concatenate((bitPlot[N_CYC:DLF*N_CYC],
                                              bitData))                       
                    if noStream % DLF == 0:    
                        plotData(gpsPlot,bitPlot*0.05)  
                        if SHOW_FRAMES:
                            edges,frameData = evalEdges(edges)
                            if len(frameData) != 0:
                                FRAME_LIST += frameData
                                plotText(FRAME_LIST)

                    deltaFreq,phaseOffset = phaseLockedLoop(gpsData)
                    dopplerPhase += phaseOffset
                    dopplerFreq  += deltaFreq       
                    
                    if codePhase >= 0:
                        COPH_LIST.append((SMP_TIME//NGPS,codePhase))
                                    
                noStream += 1
                    
            await asyncio.sleep(0)
        
    except BaseException:
        printException()
    finally:            
        MEAS_RUNNING = False
        if SAVE_RESULTS:
            saveResults(SAVE_DATE,SAT_NO)
            

# ----------- Stream live data ----------------        

async def streamLive():
    global MEAS_RUNNING
    global STATUS_MSG

    sdr = RtlSdr()
    sdr.set_bias_tee(True)
    sdr.sample_rate = SAMPLE_RATE
    if SDR_FREQCORR != 0:
        sdr.freq_correction = SDR_FREQCORR    
    sdr.center_freq = SDR_CENTERFREQ   
    sdr.gain = SDR_GAIN 

    sdrClosed = False
    measTimeout = False
    loop = asyncio.get_running_loop()
    start_time = loop.time()
    end_time = start_time+MEAS_TIME
    try:
        async for samples in sdr.stream(num_samples_or_bytes=NGPS, 
                                        format='samples'):
            pushToBuffer(samples)
            if loop.time()>end_time:                
                MEAS_RUNNING = False                    
                STATUS_MSG = 'Timeout'
            if not MEAS_RUNNING:
                STATUS_MSG = 'sdr to stop ..'
                await sdr.stop()
                STATUS_MSG = 'sdr stopped'
                sdr.close()
                sdrClosed = True
                STATUS_MSG = 'sdr closed'
    except:    
        STATUS_MSG = 'Exception from sdr.stop'
        printException()
    finally:
        MEAS_RUNNING = False
        if not sdrClosed:
            STATUS_MSG = 'sdr to close ..'
            

# --------------- code for plots ------------------        

# initial values for plot data (real and imag)
xd = np.linspace(0, DLF*N_CYC, DLF*N_CYC, endpoint=False)  # in ms
yd = np.zeros(DLF*N_CYC,dtype=MY_COMPLEX)   # global

fig,(axG,axT) = plt.subplots(1,2,figsize=(10,3.5))
fig.canvas.header_visible = False
fig.canvas.mpl_connect('key_press_event', onKeypress) 
fig.subplots_adjust(bottom=0.2)
                    
plt.subplot(1,2,1)
line1, = axG.plot(xd,yd.real,lw=.5)
line2, = axG.plot(xd,yd.imag,lw=.5)
axG.set_ylim(-0.06,0.06)
axG.set_xlabel('time (ms)')
axG.set_ylabel('amplitude (a.u.)') 

axT.set_axis_off()
W,H = 750,150                                     
xs = np.asarray([0,170,300,400])/W        
y = (H-np.asarray(range(5,175,10)))/H             
MAXL = 14    
dataL = []
keys = ['TIME (UTC)','TOW','SF-ID','SAMPLE TIME']    
for j,key in enumerate(keys):
    axT.text(xs[j],y[0],key)
for i in range(MAXL):
    dataL.append({})
    for j,key in enumerate(keys):
        dataL[i][key] = axT.text(xs[j],y[i+1],'')        
startL = axT.text(0,1.04,'')
        
# --- asyncio tasks -------------

loop = asyncio.get_event_loop()
task1 = loop.create_task(processData())
task2 = loop.create_task(streamLive())

print('Click on graph and press q to exit!')
        
